In [2]:
import requests

In [3]:
import json

In [4]:
import pandas as pd

In [91]:
import Traitement as proc

In [95]:
proc = imp.reload(proc)

On importe X et on recupere la liste des genes à mettre à jour

In [92]:
X = pd.read_pickle('clcgp/trscr.pkl')

In [ ]:
X2 = proc.update_genes(X,'temp_X.pkl')

In [17]:
array_s = X.columns.values

In [35]:
l = array_s.tolist()

In [37]:
dict_s = {s:s for s in l}

In [53]:
for v in dict_s:
    if 'HS.' in dict_s[v]:
        dict_s[v] = 'Hs'+dict_s[v][2:]
    elif '.' in dict_s[v]:
        dict_s[v] = dict_s[v][:dict_s[v].rfind('.')]

In [ ]:
X

In [54]:
dict_s["HS.9123"]

'Hs.9123'

In [55]:
dict_s_inv = {v:k for k,v in dict_s.items()}

In [57]:
dict_s_inv['Hs.9123']

'HS.9123'

In [45]:
dict_s

{'7A5': '7A5',
 'A1BG': 'A1BG',
 'A1CF': 'A1CF',
 'A26C3': 'A26C3',
 'A2BP1': 'A2BP1',
 'A2M': 'A2M',
 'A2ML1': 'A2ML1',
 'A3GALT2': 'A3GALT2',
 'A4GALT': 'A4GALT',
 'A4GNT': 'A4GNT',
 'AAA1': 'AAA1',
 'AAAS': 'AAAS',
 'AACS': 'AACS',
 'AACSL': 'AACSL',
 'AADAC': 'AADAC',
 'AADACL1': 'AADACL1',
 'AADACL2': 'AADACL2',
 'AADACL3': 'AADACL3',
 'AADACL4': 'AADACL4',
 'AADAT': 'AADAT',
 'AAK1': 'AAK1',
 'AAMP': 'AAMP',
 'AANAT': 'AANAT',
 'AARS': 'AARS',
 'AARS2': 'AARS2',
 'AARSD1': 'AARSD1',
 'AASDH': 'AASDH',
 'AASDHPPT': 'AASDHPPT',
 'AASS': 'AASS',
 'AATF': 'AATF',
 'AATK': 'AATK',
 'ABAT': 'ABAT',
 'ABCA1': 'ABCA1',
 'ABCA10': 'ABCA10',
 'ABCA11': 'ABCA11',
 'ABCA12': 'ABCA12',
 'ABCA13': 'ABCA13',
 'ABCA2': 'ABCA2',
 'ABCA3': 'ABCA3',
 'ABCA4': 'ABCA4',
 'ABCA5': 'ABCA5',
 'ABCA6': 'ABCA6',
 'ABCA7': 'ABCA7',
 'ABCA8': 'ABCA8',
 'ABCA9': 'ABCA9',
 'ABCB1': 'ABCB1',
 'ABCB10': 'ABCB10',
 'ABCB11': 'ABCB11',
 'ABCB4': 'ABCB4',
 'ABCB5': 'ABCB5',
 'ABCB6': 'ABCB6',
 'ABCB7': 'ABCB7',
 '

In [292]:
'HS.9123' in [g for g in array_s.tolist() if ('HS' in g)]

True

In [289]:
l = ['Hs'+g[2:] if 'HS.' in g else g for g in l]

In [291]:
l = [g[:g.rfind('.')] if ('.' in g) and ('Hs' not in g) else g for g in l]

In [68]:
P = pd.read_pickle('clcgp/platform-refined.pkl')

In [200]:
str1 = ', '.join(str(e) for e in l)

On recupere notre jobid

In [201]:
url = "http://epimed.univ-grenoble-alpes.fr/database/query/jobid"

In [202]:
r = requests.get(url)

In [203]:
json_l = json.loads(r.text)

In [204]:
jobid = json_l['jobid']

On envoie une requete POST pour updater une liste de gènes

In [207]:
data = {'taxid':'9606','jobid':jobid,'symbols':str1}

In [208]:
url2 = "http://epimed.univ-grenoble-alpes.fr/database/query/genes/update"

In [209]:
r2 = requests.post(url2,data)

On regarde le statut de la requete

In [293]:
url3 = "http://epimed.univ-grenoble-alpes.fr/database/query/jobstatus?jobid="+jobid

In [294]:
r3 = requests.get(url3)

In [295]:
json3 = json.loads(r3.text)

In [296]:
json3['current']

27822

In [297]:
json3

{'comment': None,
 'current': 27822,
 'jobid': '20180601172509007R170',
 'status': 'success',
 'total': 27822,
 'type': 'update'}

In [298]:
json3['status']

'success'

Possibilités pour le statut

init : travail initialisé

progress : travail en cours


succes : travail terminé

error : erreur au cours du travail

On recupere les nouveaux symboles

In [249]:
url4 = 'http://epimed.univ-grenoble-alpes.fr/database/query/jobs?jobid='+jobid

In [250]:
r4 = requests.get(url4)

In [251]:
url4

'http://epimed.univ-grenoble-alpes.fr/database/query/jobs?jobid=20180601172509007R170'

In [252]:
r4_d = r4.content.decode('utf-8')

In [253]:
csv_r = csv.reader(r4_d.splitlines(), delimiter=',')

In [254]:
f = open('temp_geneupdate.csv','w')

f.write(r4.text)

f.close()

In [5]:
df = pd.read_csv('temp_geneupdate.csv',sep=';',index_col=0)

In [6]:
S = df['gene_symbol']

In [8]:
len(S.unique())

24111

In [9]:
S.isnull().values

array([False, False, False, ..., False, False, False], dtype=bool)

In [18]:
df[S.isnull()].index

Index(['DKFZP781N1041', 'FLJ20444', 'FLJ41046', 'FLJ42842', 'FLJ44006',
       'FLJ44881', 'FLJ45256', 'FLJ45964', 'Hs.103173', 'Hs.103320',
       ...
       'LOC728302', 'LOC728454', 'LOC728912', 'LOC728939', 'LOC729837',
       'MGC27165', 'MGC34774', 'RP11-49G10', 'RPL23AP13', 'UNQ5830'],
      dtype='object', name='your_input', length=2865)

In [11]:
S[S.isnull()].to_csv('temp.csv')

In [205]:
jobid1 = '20180601140628919R521'

In [171]:
jobid2 = '20180601170610187R482'

In [206]:
jobid3 = '20180601170806155R373'

In [86]:
import numpy as np

In [89]:
28116-2865

25251

In [88]:
np.sum(S.isnull())

2865

In [81]:
len(S())

28116

In [22]:
I = df[S.isnull()].index

In [13]:
df

,tax_id,entrez,gene_symbol,title,location,chrom,status,feature,aliases,database
your_input,,,,,,,,,,
7A5,9606.0,346389.0,MACC1,"MACC1, MET transcriptional regulator",7p21.1,7,active,NaN,"[MIM:612646, Q6ZN28, OTTHUMG00000128415, MACC1...",EpiMed GENALIAS
A1BG,9606.0,1.0,A1BG,alpha-1-B glycoprotein,19q13.43,19,active,NaN,"[NM_130786, CCDS12976, P04217, OTTHUMG00000183...",EpiMed GENSYM
A1CF,9606.0,29974.0,A1CF,APOBEC1 complementation factor,10q11.23,10,active,NaN,"[APOBEC1CF, CCDS7241, ACF, ENSG00000148584, CC...",EpiMed GENSYM
A26C3,9606.0,23784.0,POTEH,POTE ankyrin domain family member H,22q11.1,22,active,NaN,"[MIM:608913, RGD:2323175, CT104.7, uc010gqp.3,...",EpiMed GENALIAS
A2BP1,9606.0,54715.0,RBFOX1,RNA binding fox-1 homolog 1,16p13.3,16,active,NaN,"[HGNC:18222, CCDS55983, Q9NWB1, MIM:605104, RB...",EpiMed GENALIAS
A2M,9606.0,2.0,A2M,alpha-2-macroglobulin,12p13.31,12,active,NaN,"[RGD:2004, NM_000014, OTTHUMG00000150267, M113...",EpiMed GENSYM
A2ML1,9606.0,144568.0,A2ML1,alpha-2-macroglobulin like 1,12p13.31,12,active,NaN,"[MIM:610627, FLJ25179, NM_144670, p170, OTTHUM...",EpiMed GENSYM
A3GALT2,9606.0,127550.0,A3GALT2,"alpha 1,3-galactosyltransferase 2",1p35.1,1,active,NaN,"[uc031plq.1, IGB3S, RGD:727913, HGNC:30005, CC...",EpiMed GENSYM
A4GALT,9606.0,53947.0,A4GALT,"alpha 1,4-galactosyltransferase (P blood group)",22q13.2,22,active,NaN,"[OTTHUMG00000150744, P1, A4GALT1, ENSG00000128...",EpiMed GENSYM


In [15]:
X.rename(columns=df['gene_symbol'],inplace=True)

NameError: name 'X' is not defined

In [71]:
df2 = df.dropna(axis=0,subset=['gene_symbol'])

df3 = df2.rename(index=dict_s_inv)

In [77]:
X2 = X.loc[:,df3.index]

X3 = X2.rename(columns=df3['gene_symbol'])

In [80]:
X3

,MACC1,A1BG,A1CF,POTEH,RBFOX1,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
S00109,5.851499,6.171907,5.918028,6.067918,5.971498,10.285974,6.176522,5.790537,8.460038,5.835065,...,7.817265,6.611084,6.065831,6.314606,6.915509,5.839582,10.095410,9.041035,7.955737,9.109739
S00132,6.032630,6.138589,5.896635,6.078204,5.820051,10.493088,5.432893,5.662889,6.644765,5.854968,...,7.287010,6.704388,6.521820,7.268500,6.740361,6.024431,10.402003,9.256530,7.725905,9.479901
S00117,6.009302,6.092017,5.803744,5.929838,5.765906,10.478976,5.866216,5.998128,7.657583,5.793922,...,7.237162,6.474971,6.061387,6.649055,6.890009,6.034414,11.359259,9.401453,7.186114,8.922180
S00126,5.983381,6.173347,5.941764,6.065766,5.838927,8.933469,5.862030,5.988594,7.401281,5.937556,...,7.804744,7.478276,5.940120,6.548390,6.814012,6.012546,10.611866,8.642759,7.643135,9.832064
S00076,5.887525,6.344101,6.061927,6.488386,6.575751,9.740659,5.664881,5.779496,7.253053,5.683528,...,7.162714,6.577942,6.033357,6.471009,6.640751,5.953102,10.423520,8.253318,7.586104,9.268958
S01288,5.853796,6.025250,5.825480,6.418713,6.158095,9.890748,5.813396,5.892707,6.850862,5.895254,...,7.684531,6.594922,6.029475,6.797182,6.830838,5.901929,10.783827,8.940287,7.717992,9.590984
S01184,6.207756,6.230184,5.966661,5.943007,5.945935,10.552859,6.413256,6.001690,6.593951,5.719485,...,7.733605,6.971865,5.957706,6.001757,6.460743,6.009907,9.916105,9.482094,7.519573,8.880927
S01089,5.978035,6.034326,6.054957,6.191227,6.061495,9.576216,5.418021,5.691004,5.903400,5.888207,...,7.586480,6.724473,6.193752,6.814486,6.277724,5.933006,9.598589,8.399235,7.004412,9.763211
S00825,5.772018,6.031219,5.724568,6.117591,5.936237,9.573255,5.885989,5.759209,6.465844,6.133646,...,7.957253,6.820651,6.274877,6.395337,7.001543,6.040936,10.984380,7.760467,7.586480,9.158971
S01125,6.399633,5.987275,5.979431,6.011093,5.802167,9.470836,6.144250,5.880979,7.067370,6.347595,...,6.886733,6.456790,6.070346,6.607848,6.412358,5.779365,10.326724,9.436639,6.493839,8.323082


In [74]:
df3['gene_symbol']

your_input
7A5            MACC1
A1BG            A1BG
A1CF            A1CF
A26C3          POTEH
A2BP1         RBFOX1
A2M              A2M
A2ML1          A2ML1
A3GALT2      A3GALT2
A4GALT        A4GALT
A4GNT          A4GNT
AAA1            AAA1
AAAS            AAAS
AACS            AACS
AACSL         AACSP1
AADAC          AADAC
AADACL1        NCEH1
AADACL2      AADACL2
AADACL3      AADACL3
AADACL4      AADACL4
AADAT          AADAT
AAK1            AAK1
AAMP            AAMP
AANAT          AANAT
AARS            AARS
AARS2          AARS2
AARSD1        AARSD1
AASDH          AASDH
AASDHPPT    AASDHPPT
AASS            AASS
AATF            AATF
              ...   
ZRANB3        ZRANB3
ZSCAN1        ZSCAN1
ZSCAN10      ZSCAN10
ZSCAN12      ZSCAN12
ZSCAN16      ZSCAN16
ZSCAN18      ZSCAN18
ZSCAN2        ZSCAN2
ZSCAN20      ZSCAN20
ZSCAN21      ZSCAN21
ZSCAN22      ZSCAN22
ZSCAN29      ZSCAN29
ZSCAN4        ZSCAN4
ZSWIM1        ZSWIM1
ZSWIM2        ZSWIM2
ZSWIM3        ZSWIM3
ZSWIM4        ZSWIM4
ZS

In [31]:
X[Iindex]

KeyError: "Index(['CICK0721Q', 'CICK0721Q', 'DJ222E13', 'DJ222E13', 'DJ222E13',\n       'DJ222E13', 'DJ222E13', 'DJ341D10', 'DJ341D10', 'DJ341D10',\n       ...\n       'Hs.92308', 'Hs.97408', 'Hs.98186', 'Hs.98197', 'Hs.98466', 'Hs.98588',\n       'Hs.98959', 'Hs.99472', 'RP11-529I10', 'RP13-360B22'],\n      dtype='object', name='your_input', length=501) not in index"

In [69]:
'HS.9123' in df2.index.values

True

In [97]:
df['gene_symbol']

your_input
7A5          MACC1
A1BG          A1BG
A1CF          A1CF
A26C3        POTEH
A2BP1       RBFOX1
A2M            A2M
A2ML1        A2ML1
A3GALT2    A3GALT2
A4GALT      A4GALT
A4GNT        A4GNT
Name: gene_symbol, dtype: object

In [121]:
X3.head()

,A1BG,7A5,A1CF,A26C3,A2BP1,A2M,A2ML1,A3GALT2,A4GALT,A4GNT
S00109,6.171907,5.851499,5.918028,6.067918,5.971498,10.285974,6.176522,5.790537,8.460038,5.835065
S00132,6.138589,6.032630,5.896635,6.078204,5.820051,10.493088,5.432893,5.662889,6.644765,5.854968
S00117,6.092017,6.009302,5.803744,5.929838,5.765906,10.478976,5.866216,5.998128,7.657583,5.793922
S00126,6.173347,5.983381,5.941764,6.065766,5.838927,8.933469,5.862030,5.988594,7.401281,5.937556
S00076,6.344101,5.887525,6.061927,6.488386,6.575751,9.740659,5.664881,5.779496,7.253053,5.683528


In [122]:
X3.rename(columns=df['gene_symbol'])

,A1BG,MACC1,A1CF,POTEH,RBFOX1,A2M,A2ML1,A3GALT2,A4GALT,A4GNT
S00109,6.171907,5.851499,5.918028,6.067918,5.971498,10.285974,6.176522,5.790537,8.460038,5.835065
S00132,6.138589,6.032630,5.896635,6.078204,5.820051,10.493088,5.432893,5.662889,6.644765,5.854968
S00117,6.092017,6.009302,5.803744,5.929838,5.765906,10.478976,5.866216,5.998128,7.657583,5.793922
S00126,6.173347,5.983381,5.941764,6.065766,5.838927,8.933469,5.862030,5.988594,7.401281,5.937556
S00076,6.344101,5.887525,6.061927,6.488386,6.575751,9.740659,5.664881,5.779496,7.253053,5.683528
S01288,6.025250,5.853796,5.825480,6.418713,6.158095,9.890748,5.813396,5.892707,6.850862,5.895254
S01184,6.230184,6.207756,5.966661,5.943007,5.945935,10.552859,6.413256,6.001690,6.593951,5.719485
S01089,6.034326,5.978035,6.054957,6.191227,6.061495,9.576216,5.418021,5.691004,5.903400,5.888207
S00825,6.031219,5.772018,5.724568,6.117591,5.936237,9.573255,5.885989,5.759209,6.465844,6.133646
S01125,5.987275,6.399633,5.979431,6.011093,5.802167,9.470836,6.144250,5.880979,7.067370,6.347595
